# Libraries

In [1]:
# My library
import sarah1 as sa
# Basic libraries
import time
import numpy as np
import pandas as pd
# Data managements
import wfdb
# Statistics
import statistics as st
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
# Special functions
from math import sqrt
# Plots
import matplotlib.pyplot as plt
# Widget
%matplotlib widget
# Theme
plt.style.use('ggplot')

In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

Using TensorFlow backend.


In [3]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Read data

In [4]:
# Patient number
patient = "00"
# Define path to data
file = "/Users/jesus.fuentes/Developer/Data/" + patient
# Read data from binary files
data = wfdb.rdrecord(file)
# Assign ECG signals
ecgs = data.p_signal
# Determine the sampling frequency
f0 = data.fs
# Read only one ECG channel
ECG = ecgs[:, 0]
# Filter ECG signal via Butterworth
ECG = sa.filterECG(ECG, sampling_rate=f0)
# Open h5 file as pandas df
peak = pd.HDFStore('../LTAF_84P_stats/LTAF_84P-' + patient + '-t_width-60-peaks.h5')
# QRS complexes
rpeaks = []
for n in range( len(peak.keys()) ):
    rpeaks.append( peak['df' + str(n)]['R'] )
# Close file to free memory
peak.close()
# Sanitise peaks
nrpeak = [sa.removeNAN(rpeaks[n]) for n in range(len(rpeaks))]

## Anotations

In [3]:
"""
label_store       symbol                                  description
0             0                              Not an actual annotation
1             1      N                                    Normal beat
2             2      L                  Left bundle branch block beat
3             3      R                 Right bundle branch block beat
4             4      a                Aberrated atrial premature beat
5             5      V              Premature ventricular contraction
6             6      F          Fusion of ventricular and normal beat
7             7      J              Nodal (junctional) premature beat
8             8      A                   Atrial premature contraction
9             9      S     Premature or ectopic supraventricular beat
10           10      E                        Ventricular escape beat
11           11      j                 Nodal (junctional) escape beat
12           12      /                                     Paced beat
13           13      Q                            Unclassifiable beat
14           14      ~                          Signal quality change
16           16      |                     Isolated QRS-like artifact
18           18      s                                      ST change
19           19      T                                  T-wave change
20           20      *                                        Systole
21           21      D                                       Diastole
22           22      "                             Comment annotation
23           23      =                         Measurement annotation
24           24      p                                    P-wave peak
25           25      B              Left or right bundle branch block
26           26      ^                      Non-conducted pacer spike
27           27      t                                    T-wave peak
28           28      +                                  Rhythm change
29           29      u                                    U-wave peak
30           30      ?                                       Learning
31           31      !                       Ventricular flutter wave
32           32      [      Start of ventricular flutter/fibrillation
33           33      ]        End of ventricular flutter/fibrillation
34           34      e                             Atrial escape beat
35           35      n                   Supraventricular escape beat
36           36      @  Link to external data (aux_note contains URL)
37           37      x             Non-conducted P-wave (blocked APB)
38           38      f                Fusion of paced and normal beat
39           39      (                                 Waveform onset
40           40      )                                   Waveform end
41           41      r       R-on-T premature ventricular contraction
"""
ann_atr = wfdb.rdann(file, 'atr', sampfrom=0, sampto=data.sig_len)
ann_qrs = wfdb.rdann(file, 'qrs', sampfrom=0, sampto=data.sig_len)
ATR = ann_atr.symbol
ATR = np.asarray(ATR)
QRS = ann_qrs.symbol
QRS = np.asarray(QRS)

NameError: name 'file' is not defined

# HRV analysis

In [5]:
# Compute intervals between R-peaks
RRI = [np.ediff1d(nrpeak[n], to_begin=0) / f0 for n in range(len(nrpeak))]
for n in range(len(RRI)):
    RRI[n][0] = np.mean(RRI[n])
# Normal-Normal (HR) and HRV
HRI = [60/RRI[n] for n in range(len(RRI))]
HRV = [np.ediff1d(HRI[n], to_begin=0) for n in range(len(HRI))]
# Basic RR features
MRR = [st.mean(RRI[n]) for n in range(len(RRI))]
VRR = [st.variance(RRI[n]) for n in range(len(RRI))]
SDRR = np.sqrt(VRR)
# Basic HRV features
MHRV = [st.mean(HRV[n]) for n in range(len(HRV))]
VHRV = [st.variance(HRV[n]) for n in range(len(HRV))]
SDHRV = np.sqrt(VHRV)

In [6]:
seed = 7
np.random.seed(seed)

In [7]:
def flat(X):
    Y = []
    for m in range(len(X)):
        for n in range(len(X[m])):
            Y.append(X[m][n])
    return Y

In [8]:
df = pd.DataFrame(MRR, columns = ['MRR'])

In [9]:
endog = df.iloc[:550].values
endog = endog.astype('float32')

## Min-Max scaler

In [10]:
scaler = MinMaxScaler(feature_range=(0, 1))
endog = scaler.fit_transform(endog)

## Train and test sets

In [11]:
nobs = len(endog)
train_size = int(nobs * 0.8)
test_size = nobs - train_size
# Split into train and test sets
train, test = endog[0:train_size, :], endog[train_size:nobs, :]
# Transform arrays into matrices
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
""" ---------------------------------------
    Reshape input (WARNING)
--------------------------------------- """
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

## LSTM model

In [12]:
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(1))
#model.compile(loss='mean_squared_error', optimizer='adam')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

2022-02-21 10:32:30.879378: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-21 10:32:30.881116: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 10. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/100
 - 3s - loss: 0.7663 - accuracy: 0.0000e+00
Epoch 2/100
 - 2s - loss: 0.6331 - accuracy: 0.0000e+00
Epoch 3/100
 - 2s - loss: 0.6291 - accuracy: 0.0000e+00
Epoch 4/100
 - 2s - loss: 0.6269 - accuracy: 0.0000e+00
Epoch 5/100
 - 2s - loss: 0.6253 - accuracy: 0.0000e+00
Epoch 6/100
 - 2s - loss: 0.6250 - accuracy: 0.0000e+00
Epoch 7/100
 - 2s - loss: 0.6246 - accuracy: 0.0000e+00
Epoch 8/100
 - 4s - loss: 0.6247 - accuracy: 0.0000e+00
Epoch 9/100
 - 3s - loss: 0.6243 - accuracy: 0.0000e+00
Epoch 10/100
 - 6s - loss: 0.6241 - accuracy: 0.0000e+00
Epoch 11/100
 - 11s - loss: 0.6234 - accuracy: 0.0000e+00
Epoch 12/100
 - 20s - loss: 0.6234 - accuracy: 0.0000e+00
Epoch 13/100
 - 22s - loss: 0.6231 - accuracy: 0.0000e+00
Epoch 14/100
 - 17s - loss: 0.6229 - accuracy: 0.0000e+00
Epoch 15/100
 - 17s - loss: 0.6226 - accuracy: 0.0000e+00
Epoch 16/100
 - 18s - loss: 0.6225 - accuracy: 0.0000e+00
Epoch 17/100
 - 20s - loss: 0.6224 - accuracy: 0.0000e+00
Epoch 18/100
 - 17s - loss: 0.622

In [13]:
from keras.models import load_model

model.save('AF_LSTM.h5')

In [14]:
del model

model = load_model('AF_LSTM.h5')

## Predictions

In [15]:
train_prediction = model.predict(trainX)
test_prediction = model.predict(testX)
# Invert train predictions
train_prediction = scaler.inverse_transform(train_prediction)
trainY = scaler.inverse_transform([trainY])
# Invert test predictions
test_prediction = scaler.inverse_transform(test_prediction)
testY = scaler.inverse_transform([testY])
# Compute RMS
print('Train Score: %.2f RMSE' % sqrt(mean_squared_error(trainY[0], train_prediction[:,0])))
print('Test Score: %.2f RMSE' % sqrt(mean_squared_error(testY[0], test_prediction[:,0])))

Train Score: 0.02 RMSE
Test Score: 0.02 RMSE


In [16]:
# Shift train predictions for plotting
trainPredictPlot = np.empty_like(endog)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back: len(train_prediction) + look_back, :] = train_prediction

# Shift test predictions for plotting
testPredictPlot = np.empty_like(endog)
testPredictPlot[:,:] = np.nan
testPredictPlot[len(train_prediction) + (2*look_back) + 1: nobs - 1, :] = test_prediction

## Plots

In [17]:
fig = plt.figure(figsize=(8, 5), constrained_layout=True)
plt.plot(scaler.inverse_transform(endog), 'o', color='#5e6162', lw=3, alpha=0.7, label='Real data')
plt.plot(trainPredictPlot, color='#3399ff', lw=3, alpha=0.7, label='Training set')
plt.plot(testPredictPlot, color='#ff69b4', lw=3, alpha=0.7, label='Forecasting')
plt.legend(loc='best')

plt.xlabel('Time', fontsize=16)
plt.ylabel('Mean RR (s)', fontsize=16)
plt.title('HR oscillations from AF-patient', fontsize=18, weight='bold')
#plt.xticks(tr, labels=tt, rotation=45)
#plt.locator_params(axis="x", nbins=15)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …